# Robust Portfolio

This tutorial shows how to construct and solve the Robust Portfolio problem, introduced by Bertsimas and Sim 2004 [1] (https://pubsonline.informs.org/doi/abs/10.1287/opre.1030.0065)
We show to how solve the simple portfolio problem (Section 6.2). Using the same formulation as in [1], the problem can be formulated as follows:
$$
\begin{align}
\max_{\mathbf{x}} &\;\mathbf{\tilde{p}}^T \mathbf{x} \nonumber \\

\text{s.t. } &\mathbf{1}^T\mathbf{x} = 1 \nonumber \\

&\mathbf{x}\geq \mathbf{0} \nonumber \\

&\mathbf{p}-\pmb{\sigma} \leq \mathbf{\tilde{p}} \leq \mathbf{p}+\pmb{\sigma} \\
\end{align}
$$

where there are $n$ stocks, the unknown return of the stocks is denoted by $\mathbf{\tilde{p}}$, and $\mathbf{x}$ denotes the fraction fo wealth invested in the stocks. The expected return of the stocks are denoted by $\mathbf{p}$, and their uncertainties are captured by $\pmb{\sigma}$.

In [2]:
import cvxpy as cp
import numpy as np

from lropt import Box
from lropt.robust_problem import RobustProblem
from lropt.uncertain import UncertainParameter

np.random.seed(seed=1234)

We start by defining the relevant constants. We use the same values as given in the example in Section 6.2 of [1].

In [3]:
n = 150 #Number of stocks
i = np.arange(1, n+1) #Indices vector
p = 1.15 + i*0.05/150 #Mean returns vector
sigma = np.array(0.05/450 * (2*i*n*(n+1))**0.5) #Uncertainty vector
Gamma = 5 #Budget TODO: Do we need this?

The uncertain parameter $\mathbf{\tilde{p}}$ is formulated using LROPT in the block below. We use the box uncertainty set, which is defined as follows:

Box uncertainty: $\{ u \mid\| Au + b\|_\infty \leq \rho \}$ 

- $\rho$ : float, optional  
  * Box scaling. Default 1.0.
- $A$ : np.array, optional
  * Scaling matrix for u. Default identity matrix.
- $b$ : np.array, optional
  * Relocation vector for u. Default None.

We define a helper uncertain variable $\mathbf{u}$ that satisfies $-\pmb{\sigma}\leq\mathbf{u}\leq\pmb{\sigma}$, so the uncertain paramater $\mathbf{\tilde{p}}$ is defined as $\mathbf{\tilde{p}} = \mathbf{p} + \mathbf{u}$.

In [12]:
#Define a helper uncertainty variable u
uncertainty_set = Box(rho=1, a=np.diag(1/sigma))
u = UncertainParameter(n, uncertainty_set=uncertainty_set)
p_hat = p+u
x = cp.Variable(n) #Optimization variable

#Define and solve the problem
objective = cp.Maximize(p_hat@x)
constraints = [
                cp.sum(x)==1,
                x>=0,
              ]
prob = RobustProblem(objective=objective, constraints=constraints)
prob.solve(solver="CLARABEL")

1.1266846798960153